# 数据库模型models 与 类型验证模型 schemas 的设计原则与运用范围

## models 
1. 考虑所有需要保存到数据库中的属性，不包括暂存属性和转换储存前的属性，比如存储用户密码为hased_password 而不是 password,考虑要*全*
2. 包括通过API与用户交互的属性
3. 如果没有通过API获取，可以在model中设置默认值，或者在CRUD创建时添加 > user_crud.create() 
4. sqlalchemy 中 多重继承有问题 , 建议直接 都从 Base 模型继承 ，__tablename__ 的设置可以手动，也可以@declared_attr
5. 考虑数据库如mysql的类型，sqlalchemy 中的类型，编程语言如python的类型的不同，参见相关表格，特别注意datetime
6. 在设置类型时注意 datetime.now 与 datetime.now() 的区别如：create_time= Column(DateTime,default=datetime.now) 
   
### 一对多

### 一对一

### 多对多

In [1]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column,String,Integer,Boolean,DateTime,ForeignKey
from sqlalchemy.orm import relationship


Base = declarative_base()

class User(Base):

    __tablename__ = "user"
        
    id = Column(Integer,primary_key=True,index=True)       #一般系统自建不需要用户schema
    username = Column(String(30),index=True)
    hashed_password = Column(String(500),nullable=False)    # 用户密码是转换为hashed_password，用户schema 为password
    email = Column(String(30),nullable=False)
    telephone = Column(Integer,index=True)
    tax = Column(Integer,index=True)
    useraddress = Column(String(100))
    content = Column(String(500))

    is_active = Column(Boolean(),default=True)            # 普通用户可能不需要知道，但系统需要储存的，都应该放到数据库属性中
    is_shuser = Column(Boolean(),default=False)
    is_fcuser = Column(Boolean(),default=False)
    is_manager = Column(Boolean(),default=False)    
    
    create_time= Column(DateTime,default=datetime.now)     #  系统自动储存，一般不需要用户schema
    update_time= Column(DateTime,default=datetime.now,onupdate=datetime.now)   

## schemas 
1. 类型验证，主要考虑的是面向各个对象属性来路和去路的控制性手段。
2. 分为 变量的类型控制 var:Schema 如：current_user: models.User = Depends(deps.get_current_active_user)） 
3. 和 api 中 用户输入控制：
    @router.put("/me", response_model=schemas.User)   
    def update_user_me(...):pass
4. 一般共享的Base类，主要是指 在数据库储存、API中都具有的属性，所以一般不包含id,password/hased_password等
5. 一般Create类，考虑password，以及一些在Base里不强制可选Optional，但Create强制提供
6. 一般Update类，跟Base 类同，主要考虑哪些是更新有，而Base没有
7. 一般InDBBase类，跟返回用户的Item类属性同
8. 一般InDB类，在InDBBase基础上添加hashpassword


In [ ]:
from typing import Optional
from pydantic import BaseModel, EmailStr

# Shared properties  共享的属性
class UserBase(BaseModel):

    username : Optional[str] = None
    # password not shared
    email: Optional[EmailStr] = None
    telephone: Optional[int] = None
    tax : Optional[str] = None
    useraddress: Optional[str] = None
    content: Optional[str] = None    


# Properties to receive via API on creation  新建user时，通过API接收的属性  属性来源之一，用户
class UserCreate(UserBase):
    username: str
    password: str
    email : EmailStr


# Properties to receive via API on update   更新user时，通过API接收的属性
class UserUpdate(UserBase):
    password: Optional[str] = None


class UserInDBBase(UserBase):
    id: Optional[int] = None

    class Config:
        orm_mode = True


# Additional properties to return via API   通过API返回给frontend/client的属性,额外属性，一般同时也是数据库的属性
class User(UserInDBBase):   # the same as UserInDBBase,一般同时也是数据库的属性(除密码)
    pass


# Additional properties stored in DB  储存进数据库的最终数据，一般包含没有通过API 获得，但需要储存的数据 
class UserInDB(UserInDBBase):
    hashed_password: str
    is_active: Optional[bool] = True
    is_shuser: Optional[bool] = False
    is_fcuser: Optional[bool] = False
    is_manager: Optional[bool] = False